Character-level RNN Language Model written in Tensorflow.

The purpose of this notebook is to demonstrate how we could build Recurrent Neural Network for character language modeling and train them on the George R.R. Martin's Game of Thrones novel series. We will show how this learned language model to able to generate sequences of character that constitute story text with similar style as the GoT's novel text.

To read:
- Tensorflow implementation of dynamic rnn, sequence loss by example

In [1]:
# import all required libraries
import numpy as np
import tensorflow as tf

import collections
import time

In [2]:
# define a function to load and preprocess the text corpus then return list of chars
def read_file(path):
    with open(corpus_path) as f:
        char_tokens = ['*STOP*']
        text = f.read()
        char_tokens.extend(text)
        
        for i in range(len(char_tokens)):
            if char_tokens[i] == '\n':
                char_tokens[i] = '*STOP*'
        
        return char_tokens

In [3]:
def build_dataset(tokens):
    counts = []
    counts.extend(collections.Counter(tokens).most_common())
    
    dictionary = dict()
    data = list()
    
    for token, _ in counts:
        dictionary[token] = len(dictionary)
        
    for token in tokens:
        data.append(dictionary[token])
        
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return data, dictionary, reverse_dictionary

In [4]:
def generate_batch(dataset, batch_size, num_steps, offset=0):
    assert offset + batch_size * num_steps < len(dataset)
    
    batch_context = np.ndarray((batch_size, num_steps), dtype=np.int32)
    batch_target = np.ndarray((batch_size, num_steps), dtype=np.int32)
    
    for i in range(batch_size):
        batch_context[i] = dataset[offset : offset+num_steps]
        batch_target[i] = dataset[offset+1 : offset+num_steps+1]
        offset += num_steps
        
    return batch_context, batch_target, offset

In [5]:
# define parameters of the program
corpus_path = '../data/got_all_edited.txt'

num_epoch = 30

batch_size = 30
num_steps = 60
embedding_size = 100

hidden_unit_size = 256
vocabulary_size = 20000
learning_rate = 1e-4

In [6]:
tokens = read_file(corpus_path)
data, tokendict, tokendictreversed = build_dataset(tokens)

vocabsize = len(tokendict)

In [7]:
train, label, _ = generate_batch(data, 5, num_steps)
for batch_train, batch_label in zip(train, label):
    print ''.join([tokendictreversed[token] for token in batch_train]) + ' --> '
    print ''.join([tokendictreversed[word] for word in batch_label])
    print '----------'

*STOP*"We should start back," Gared urged as the woods began to g --> 
"We should start back," Gared urged as the woods began to gr
----------
row dark around them. "The wildlings are dead."*STOP*"Do the dead --> 
ow dark around them. "The wildlings are dead."*STOP*"Do the dead 
----------
 frighten you?" Ser Waymar Royce asked with just the hint of --> 
frighten you?" Ser Waymar Royce asked with just the hint of 
----------
 a smile.*STOP*Gared did not rise to the bait. He was an old man, --> 
a smile.*STOP*Gared did not rise to the bait. He was an old man, 
----------
 past fifty, and he had seen the lordlings come and go. "Dea --> 
past fifty, and he had seen the lordlings come and go. "Dead
----------


In [ ]:
graph = tf.Graph()
with graph.as_default():
    # setup input and labels placeholders
    train_inputs = tf.placeholder(tf.int32, shape=[None, num_steps])
    train_labels = tf.placeholder(tf.int32, shape=[None, num_steps])
    keep_prob = tf.placeholder(tf.float32)
    bsize = tf.placeholder(tf.int32)
    
    # instantiate embedding matrix
    charvectors = tf.Variable(tf.random_normal([vocabsize, embedding_size]))
    charvector = tf.nn.embedding_lookup(charvectors, train_inputs)
    charvector = tf.nn.dropout(charvector, keep_prob)
    
    # define the rnn cell and the initial state
    rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_unit_size, forget_bias=0.0, state_is_tuple=True)
    init_state = rnn_cell.zero_state(bsize, tf.float32)
    rnn_outputs, final_state = tf.nn.dynamic_rnn(rnn_cell, charvector, initial_state=init_state)   
        
    # reshape the outputs into 2D vectors
    rnn_outputs = tf.reshape(rnn_outputs, [bsize * num_steps, hidden_unit_size])
     
    logits_weights = tf.Variable(tf.truncated_normal([hidden_unit_size, vocabsize], stddev=0.1))
    logits_biases = tf.Variable(tf.zeros([vocabsize]))
    logits = tf.matmul(rnn_outputs, logits_weights) + logits_biases
    
    loss_weights = tf.ones([batch_size * num_steps])
    losses = tf.nn.seq2seq.sequence_loss_by_example([logits], [train_labels], [loss_weights])
    loss = tf.reduce_sum(losses) / batch_size
        
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    
    var_saver = tf.train.Saver(tf.trainable_variables())
    path = 'checkpoints/char_rnn_langmodel.ckpt'
    start_time = time.time()
    
    for epoch in range(num_epoch):
        offset = 0
        total_loss = 0.
        iters = 0
        seqs = 0
        training_state = sess.run(init_state, feed_dict={bsize: batch_size})
        while offset + batch_size * num_steps < len(data):
            batch_context, batch_target, offset = generate_batch(data, batch_size, num_steps, offset)
            feed_dict = {train_inputs: batch_context, train_labels: batch_target, 
                         keep_prob: .5, init_state:training_state,
                         bsize: batch_size}
            training_loss, training_state, _ = sess.run([loss, final_state, train_step], feed_dict=feed_dict)
            
            total_loss += training_loss
            iters += num_steps
        
            seqs += batch_size
        
            if seqs % 1000 == 0:
                perplexity = np.exp(total_loss / iters)
                print 'Epoch: %d, Time elapsed: %.2f s, Tokens trained: %04d, Perplexity: %.4f' % \
                    (epoch+1, (time.time() - start_time), offset, perplexity)
                    
        # save checkpoint every 10000 train steps
        checkpoint_path = var_saver.save(sess, path)
        print 'Epoch completed. Checkpoint saved as: %s' % (checkpoint_path)

Epoch: 1, Time elapsed: 5.29 s, Tokens trained: 180000, Perplexity: 45.0148
Epoch: 1, Time elapsed: 10.33 s, Tokens trained: 360000, Perplexity: 27.4767
Epoch: 1, Time elapsed: 15.39 s, Tokens trained: 540000, Perplexity: 21.8503
Epoch: 1, Time elapsed: 20.38 s, Tokens trained: 720000, Perplexity: 18.9705
Epoch: 1, Time elapsed: 25.38 s, Tokens trained: 900000, Perplexity: 17.2149
Epoch: 1, Time elapsed: 30.42 s, Tokens trained: 1080000, Perplexity: 15.9540
Epoch: 1, Time elapsed: 35.43 s, Tokens trained: 1260000, Perplexity: 15.0330
Epoch: 1, Time elapsed: 40.49 s, Tokens trained: 1440000, Perplexity: 14.3094
Epoch: 1, Time elapsed: 45.72 s, Tokens trained: 1620000, Perplexity: 13.7029
Epoch: 1, Time elapsed: 50.86 s, Tokens trained: 1800000, Perplexity: 13.2236
Epoch: 1, Time elapsed: 55.92 s, Tokens trained: 1980000, Perplexity: 12.8161
Epoch: 1, Time elapsed: 61.07 s, Tokens trained: 2160000, Perplexity: 12.4699
Epoch: 1, Time elapsed: 66.13 s, Tokens trained: 2340000, Perplexity: 